## 1. Import and install neccessary libraries

In [ ]:
import json
import matplotlib
matplotlib.use("Agg") 
import matplotlib.pyplot as plt

In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2
!pip install jiwer
!pip install einops addict easydict

## 2. Download base model

In [2]:
from huggingface_hub import snapshot_download
snapshot_download("unsloth/DeepSeek-OCR", local_dir = "deepseek_ocr")

Fetching 21 files:   0%|          | 0/21 [00:00<?, ?it/s]

README-checkpoint.md: 0.00B [00:00, ?B/s]

assets/show2.jpg:   0%|          | 0.00/216k [00:00<?, ?B/s]

assets/show1.jpg:   0%|          | 0.00/117k [00:00<?, ?B/s]

LICENSE: 0.00B [00:00, ?B/s]

...

processor_config.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

'/kaggle/working/deepseek_ocr'

In [3]:
from unsloth import FastVisionModel # FastLanguageModel for LLMs
import torch
from transformers import AutoModel
import os
os.environ["UNSLOTH_WARN_UNINITIALIZED"] = '0'
# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Qwen3-VL-8B-Instruct-bnb-4bit", # Qwen 3 vision support
    "unsloth/Qwen3-VL-8B-Thinking-bnb-4bit",
    "unsloth/Qwen3-VL-32B-Instruct-bnb-4bit",
    "unsloth/Qwen3-VL-32B-Thinking-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastVisionModel.from_pretrained(
    "./deepseek_ocr",
    load_in_4bit = True, # Use 4bit to reduce memory use. False for 16bit LoRA.
    auto_model = AutoModel,
    trust_remote_code=True,
    unsloth_force_compile=True,
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-12-19 08:44:14.619787: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766133854.836593      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766133854.891076      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and can yield errors.


Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.12.7: Fast Deepseekocr patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and can yield errors.
You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and can yield errors.
Some weights of DeepseekOCRForCausalLM were not initialized from the model checkpoint at ./deepseek_ocr and are newly initialized: ['model.vision_model.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 3. Creating test data

In [4]:
# @title Create test set
import os
import json
import random
from jiwer import wer, cer
from PIL import Image
import torch

FastVisionModel.for_inference(model)

TEST_ROOT = "/kaggle/input/uit-hwdb-dataset"
TEST_FOLDERS = [
    "UIT_HWDB_line/UIT_HWDB_line/test_data",
    "UIT_HWDB_paragraph/UIT_HWDB_paragraph/test_data",
    "UIT_HWDB_word/UIT_HWDB_word/test_data",
]

PROMPT = "<image>\nFree OCR. "


def sample_test_images(root_dir, max_folders=20, max_per_folder=5):
    samples = []

    subfolders = [
        d for d in os.listdir(root_dir)
        if os.path.isdir(os.path.join(root_dir, d))
    ]

    subfolders = random.sample(subfolders, min(max_folders, len(subfolders)))

    for sub in subfolders:
        sub_path = os.path.join(root_dir, sub)
        label_path = os.path.join(sub_path, "label.json")

        if not os.path.exists(label_path):
            continue

        with open(label_path, "r", encoding="utf-8") as f:
            labels = json.load(f)

        for img_name, gt in labels.items():
            img_path = os.path.join(sub_path, img_name)

            # ===== FIX HERE =====
            if (
                not isinstance(gt, str)
                or not gt.strip()
                or not img_name
                or not os.path.exists(img_path)
            ):
                continue
            # ===================

            samples.append({
                "image": img_path,
                "gt": gt.strip()
            })

            if len(samples) >= max_folders * max_per_folder:
                return samples

    return samples

In [5]:
test_data = []
for f in TEST_FOLDERS:
    path = os.path.join(TEST_ROOT, f)
    samples = sample_test_images(path)
    test_data.extend(samples)

test_data[:5]
print("Total test data: ", len(test_data))

Total test data:  231


In [6]:
from datasets import Dataset
from PIL import Image

def build_eval_dataset(samples):
    return Dataset.from_dict({
        "image_path": [Image.open(s["image"]).convert("RGB") for s in samples],
        "text": [s["gt"] for s in samples]
    })

In [7]:
eval_dataset = build_eval_dataset(test_data)

## 4. Evaluate Base Model

In [8]:
# @title Create evaluation functions

import json
import os
from typing import Dict
import numpy as np
from jiwer import cer
from tqdm import tqdm
from datasets import load_dataset


def calculate_cer(ref: str, hyp: str) -> float:
    """Helper to calculate CER and convert to percentage."""
    return cer(ref, hyp) * 100


def evaluate_model(
    model,
    tokenizer,
    dataset,
    num_samples: int = 100,
    base_size: int = 1024,
    image_size: int = 640,
    crop_mode: bool = True,
    verbose: bool = True
):
    """
    Runs the model over a subset of the dataset to see how it performs.
    It'll calculate CER stats and save all the predictions.
    """

    results = {
        'cer_scores': [],
        'predictions': [],
        'references': [],
        'sample_indices': []
    }

    # make sure we don't try to sample more than we have
    num_samples = min(num_samples, len(dataset))

    # Grab evenly spaced samples from the dataset
    indices = np.linspace(0, len(dataset) - 1, num_samples, dtype=int)

    # Use tqdm for a progress bar if verbose
    iterator = tqdm(indices, desc="Evaluating") if verbose else indices

    for idx in iterator:
        sample = dataset[int(idx)]

        # The model.infer method needs a file path, so we save a temp image
        temp_image_path = f"temp_eval_image_{idx}.jpg"
        sample['image_path'].save(temp_image_path)

        prediction = ""
        reference = sample["text"].strip()

        try:
            # Run the actual inference
            prediction = model.infer(
                tokenizer,
                prompt="<image>\nFree OCR. ",
                image_file=temp_image_path,
                output_path="temp_output",
                base_size=base_size,
                image_size=image_size,
                crop_mode=crop_mode,
                eval_mode=True,
                save_results=False,
                test_compress=False
            )

            prediction = prediction.strip()

            # Calculate CER
            cer_score = calculate_cer(reference, prediction)

            results['cer_scores'].append(cer_score)
            results['predictions'].append(prediction)
            results['references'].append(reference)
            results['sample_indices'].append(int(idx))

        except Exception as e:
            # Don't let one bad sample crash the whole evaluation
            print(f"\nError processing sample {idx}: {e}")
            print(f"Reference was: {reference}")
            continue
        finally:
            # Clean up the temp file whether it succeeded or failed
            if os.path.exists(temp_image_path):
                os.remove(temp_image_path)

    # Add summary stats
    if results['cer_scores']:
        results['mean_cer'] = np.mean(results['cer_scores'])
        results['median_cer'] = np.median(results['cer_scores'])
        results['std_cer'] = np.std(results['cer_scores'])
        results['min_cer'] = np.min(results['cer_scores'])
        results['max_cer'] = np.max(results['cer_scores'])
    else:
        print("Warning: No samples were successfully processed.")
        results['mean_cer'] = -1.0

    results['num_samples'] = len(results['cer_scores'])

    return results

def print_evaluation_summary(results: Dict, title: str = "Evaluation Results"):
    """Prints a nice summary of the stats to the console."""

    print("\n" + "="*60)
    print(f"{title}")
    print("="*60)
    print(f"Number of samples: {results['num_samples']}")
    print(f"Mean CER: {results['mean_cer']:.2f}%")
    print(f"Median CER: {results['median_cer']:.2f}%")
    print(f"Std Dev: {results['std_cer']:.2f}%")
    print(f"Min CER: {results['min_cer']:.2f}%")
    print(f"Max CER: {results['max_cer']:.2f}%")
    print("="*60)

    # Show best and worst examples
    sorted_indices = np.argsort(results['cer_scores'])

    print("\n Best Predictions (Lowest CER):")
    for i in range(min(3, len(sorted_indices))):
        idx = sorted_indices[i]
        print(f"\nSample {results['sample_indices'][idx]} (CER: {results['cer_scores'][idx]:.2f}%)")
        print(f"Reference:  {results['references'][idx][:100]}...")
        print(f"Prediction: {results['predictions'][idx][:100]}...")

    print("\n Worst Predictions (Highest CER):")
    for i in range(min(3, len(sorted_indices))):
        idx = sorted_indices[-(i+1)]
        print(f"\nSample {results['sample_indices'][idx]} (CER: {results['cer_scores'][idx]:.2f}%)")
        print(f"Reference:  {results['references'][idx][:100]}...")
        print(f"Prediction: {results['predictions'][idx][:100]}...")

def save_evaluation_results(results: Dict, filepath: str):
    """Save full results dictionary to a JSON file."""
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    print(f"\n Results saved to {filepath}")

In [9]:
print("Loading evaluation dataset...")
from datasets import load_dataset

print("\n Running Baseline Evaluation...")
baseline_results = evaluate_model(
    model=model,
    tokenizer=tokenizer,
    dataset=eval_dataset,
    num_samples=200,
    base_size=1024,
    image_size=640,
    crop_mode=True,
    verbose=True
)

print_evaluation_summary(baseline_results, "Baseline Model Performance")
save_evaluation_results(baseline_results, "baseline_evaluation.json")

Loading evaluation dataset...

 Running Baseline Evaluation...


Evaluating:   0%|          | 0/200 [00:00<?, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])


Evaluating:   0%|          | 1/200 [00:12<42:47, 12.90s/it]

BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])


...

BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([3, 100, 1280])


Evaluating:  56%|█████▋    | 113/200 [15:11<28:07, 19.40s/it]

BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([4, 100, 1280])


Evaluating: 100%|██████████| 200/200 [20:28<00:00,  6.14s/it]


Baseline Model Performance
Number of samples: 200
Mean CER: 810.95%
Median CER: 60.13%
Std Dev: 4645.85%
Min CER: 6.06%
Max CER: 61100.00%

 Best Predictions (Lowest CER):

Sample 84 (CER: 6.06%)
Reference:  nghiệp nhà nước và việc tổ chức sắp xếp lại doanh nghiệp nhà nước....
Prediction: nghịp nhà nước và việc tổ chức, sắp xếp lời doanh nghiệp nhà nước....

Sample 45 (CER: 7.79%)
Reference:  Ngay cả trường hợp ta không qui định, người dân và doanh nghiệp vẫn thực hiện...
Prediction: Ngay cũ trường hẹp tư liêng qui định, người dân và doanh nghiệp vẫn thực hiện...

Sample 41 (CER: 8.45%)
Reference:  chúng ta cần thay đổi trong quan điểm, không nên để hệ thống hành chính...
Prediction: chúng ta còn thay đổi trong quan điểm, không nên đọc thông hành chính...

 Worst Predictions (Highest CER):

Sample 177 (CER: 61100.00%)
Reference:  11...
Prediction: {
 "Line": {
  "line": [
   "(-7.0, -0.0, 0.0, 0.0)",
   "(-7.0, -0.0, 0.0, 0.0)",
   "(-7.0, -0.0, ...

Sample 149 (CER: 13400.00%)
Refere

## 5. Fine-tuning Deepseek OCR

In [10]:
model = FastVisionModel.get_peft_model(
    model,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],

    r = 16,           # The larger, the higher the accuracy, but might overfit
    lora_alpha = 16,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
    # target_modules = "all-linear", # Optional now! Can specify a list if needed
)

Unsloth: Making `model.base_model.model.model` require gradients


### Preparing training data

In [11]:
# @title Create train dataset

import json
import random

def sample_from_subfolders(
    root_dir,
    max_folders=200,     # số subfolder (writer) lấy
    max_per_folder=20     # số ảnh mỗi subfolder
):
    """
    Trả về list (img_path, text, subfolder_id)
    """
    samples = []

    subfolders = sorted([
        d for d in os.listdir(root_dir)
        if os.path.isdir(os.path.join(root_dir, d))
    ])

    if max_folders is not None:
        subfolders = random.sample(
            subfolders,
            min(max_folders, len(subfolders))
        )

    for sub in subfolders:
        sub_path = os.path.join(root_dir, sub)
        label_path = os.path.join(sub_path, "label.json")

        if not os.path.exists(label_path):
            continue

        # Load labels của subfolder này
        with open(label_path, "r", encoding="utf-8") as f:
            labels = json.load(f)

        # Chỉ lấy ảnh có trong labels.json
        images = [
            img for img in labels.keys()
            if os.path.exists(os.path.join(sub_path, img))
        ]
        
        if not images:
            continue

        chosen_imgs = random.sample(
            images,
            min(max_per_folder, len(images))
        )

        for img_name in chosen_imgs:
            text = labels[img_name].strip()
            if not text:
                continue

            img_path = os.path.join(sub_path, img_name)
            samples.append(
                {
                    "image": img_path,
                    "text" : text,
                    "sub" : sub
                }
            )
    return samples


In [12]:
root_dir = "/kaggle/input/uit-hwdb-dataset"
sub_folders = ["UIT_HWDB_line/UIT_HWDB_line", "UIT_HWDB_paragraph/UIT_HWDB_paragraph", "UIT_HWDB_word/UIT_HWDB_word"]
samples = []
for sub_fold in sub_folders:
    print("Processing ", sub_fold)
    path = os.path.join(root_dir, sub_fold)
    folder_path = os.path.join(path, "train_data")
    sample = sample_from_subfolders(folder_path)
    samples.extend(sample)

Processing  UIT_HWDB_line/UIT_HWDB_line
Processing  UIT_HWDB_paragraph/UIT_HWDB_paragraph
Processing  UIT_HWDB_word/UIT_HWDB_word


In [13]:
print(samples[:5])
print("Total samples: ", len(samples))

[{'image': '/kaggle/input/uit-hwdb-dataset/UIT_HWDB_line/UIT_HWDB_line/train_data/111/8.jpg', 'text': 'gà và ham gà chọi từ ngày còn " thò lò mũi xanh ", đến nay hắn đã 41', 'sub': '111'}, {'image': '/kaggle/input/uit-hwdb-dataset/UIT_HWDB_line/UIT_HWDB_line/train_data/111/3.jpg', 'text': 'trận được tính làm 10 hồ - hiệp đấu, mỗi hồ 15 phút), cặp gà vẫn chưa phân thắng', 'sub': '111'}, {'image': '/kaggle/input/uit-hwdb-dataset/UIT_HWDB_line/UIT_HWDB_line/train_data/111/10.jpg', 'text': 'bạc gà lão luyện của vùng Bắc bộ. Chưa trận nào hắn chơi dưới 2 triệu, trận nào', 'sub': '111'}, {'image': '/kaggle/input/uit-hwdb-dataset/UIT_HWDB_line/UIT_HWDB_line/train_data/111/5.jpg', 'text': '" cụt " tỏ ra rất vui vì đã dốc gần 5 triệu cho trận đá này.', 'sub': '111'}, {'image': '/kaggle/input/uit-hwdb-dataset/UIT_HWDB_line/UIT_HWDB_line/train_data/111/9.jpg', 'text': 'tuổi. Gần 30 năm lăn lộn trong nghề đã khiến hắn trở thành một tay', 'sub': '111'}]
Total samples:  8537


In [14]:
from PIL import Image

instruction = "<image>\nFree OCR. "

def convert_to_conversation(sample):
    """Convert dataset sample to conversation format"""
    img_path = sample['image']
    image = Image.open(img_path).convert("RGB")
    conversation = [
        {
            "role": "<|User|>",
            "content": instruction,
            "images": [image]
        },
        {
            "role": "<|Assistant|>",
            "content": sample["text"]
        },
    ]
    return {"messages": conversation}

# Load dataset

# dataset = load_dataset("hezarai/parsynth-ocr-200k", split = "train[:1000]")
# dataset = dataset.rename_column("image_path", "image")

In [15]:
converted_dataset = [convert_to_conversation(sample) for sample in samples]

In [16]:
print(converted_dataset[0])

{'messages': [{'role': '<|User|>', 'content': '<image>\nFree OCR. ', 'images': [<PIL.Image.Image image mode=RGB size=2048x109 at 0x7E608EB5F0D0>]}, {'role': '<|Assistant|>', 'content': 'gà và ham gà chọi từ ngày còn " thò lò mũi xanh ", đến nay hắn đã 41'}]}


In [17]:
# @title Create datacollator

import torch
import math
from dataclasses import dataclass
from typing import Dict, List, Any, Tuple
from PIL import Image, ImageOps
from torch.nn.utils.rnn import pad_sequence
import io

from deepseek_ocr.modeling_deepseekocr import (
    format_messages,
    text_encode,
    BasicImageTransform,
    dynamic_preprocess,
)

@dataclass
class DeepSeekOCRDataCollator:
    """
    Args:
        tokenizer: Tokenizer
        model: Model
        image_size: Size for image patches (default: 640)
        base_size: Size for global view (default: 1024)
        crop_mode: Whether to use dynamic cropping for large images
        train_on_responses_only: If True, only train on assistant responses (mask user prompts)
    """
    tokenizer: Any
    model: Any
    image_size: int = 640
    base_size: int = 1024
    crop_mode: bool = True
    image_token_id: int = 128815
    train_on_responses_only: bool = True

    def __init__(
        self,
        tokenizer,
        model,
        image_size: int = 640,
        base_size: int = 1024,
        crop_mode: bool = True,
        train_on_responses_only: bool = True,
    ):
        self.tokenizer = tokenizer
        self.model = model
        self.image_size = image_size
        self.base_size = base_size
        self.crop_mode = crop_mode
        self.image_token_id = 128815
        self.dtype = model.dtype  # Get dtype from model
        self.train_on_responses_only = train_on_responses_only

        self.image_transform = BasicImageTransform(
            mean=(0.5, 0.5, 0.5),
            std=(0.5, 0.5, 0.5),
            normalize=True
        )
        self.patch_size = 16
        self.downsample_ratio = 4

        # Get BOS token ID from tokenizer
        if hasattr(tokenizer, 'bos_token_id') and tokenizer.bos_token_id is not None:
            self.bos_id = tokenizer.bos_token_id
        else:
            self.bos_id = 0
            print(f"Warning: tokenizer has no bos_token_id, using default: {self.bos_id}")

    def deserialize_image(self, image_data) -> Image.Image:
        """Convert image data (bytes dict or PIL Image) to PIL Image in RGB mode"""
        if isinstance(image_data, Image.Image):
            return image_data.convert("RGB")
        elif isinstance(image_data, dict) and 'bytes' in image_data:
            image_bytes = image_data['bytes']
            image = Image.open(io.BytesIO(image_bytes))
            return image.convert("RGB")
        else:
            raise ValueError(f"Unsupported image format: {type(image_data)}")

    def calculate_image_token_count(self, image: Image.Image, crop_ratio: Tuple[int, int]) -> int:
        """Calculate the number of tokens this image will generate"""
        num_queries = math.ceil((self.image_size // self.patch_size) / self.downsample_ratio)
        num_queries_base = math.ceil((self.base_size // self.patch_size) / self.downsample_ratio)

        width_crop_num, height_crop_num = crop_ratio

        if self.crop_mode:
            img_tokens = num_queries_base * num_queries_base + 1
            if width_crop_num > 1 or height_crop_num > 1:
                img_tokens += (num_queries * width_crop_num + 1) * (num_queries * height_crop_num)
        else:
            img_tokens = num_queries * num_queries + 1

        return img_tokens

    def process_image(self, image: Image.Image) -> Tuple[List, List, List, List, Tuple[int, int]]:
        """
        Process a single image based on crop_mode and size thresholds

        Returns:
            Tuple of (images_list, images_crop_list, images_spatial_crop, tokenized_image, crop_ratio)
        """
        images_list = []
        images_crop_list = []
        images_spatial_crop = []

        if self.crop_mode:
            # Determine crop ratio based on image size
            if image.size[0] <= 640 and image.size[1] <= 640:
                crop_ratio = (1, 1)
                images_crop_raw = []
            else:
                images_crop_raw, crop_ratio = dynamic_preprocess(
                    image, min_num=2, max_num=9,
                    image_size=self.image_size, use_thumbnail=False
                )

            # Process global view with padding
            global_view = ImageOps.pad(
                image, (self.base_size, self.base_size),
                color=tuple(int(x * 255) for x in self.image_transform.mean)
            )
            images_list.append(self.image_transform(global_view).to(self.dtype))

            width_crop_num, height_crop_num = crop_ratio
            images_spatial_crop.append([width_crop_num, height_crop_num])

            # Process local views (crops) if applicable
            if width_crop_num > 1 or height_crop_num > 1:
                for crop_img in images_crop_raw:
                    images_crop_list.append(
                        self.image_transform(crop_img).to(self.dtype)
                    )

            # Calculate image tokens
            num_queries = math.ceil((self.image_size // self.patch_size) / self.downsample_ratio)
            num_queries_base = math.ceil((self.base_size // self.patch_size) / self.downsample_ratio)

            tokenized_image = ([self.image_token_id] * num_queries_base + [self.image_token_id]) * num_queries_base
            tokenized_image += [self.image_token_id]

            if width_crop_num > 1 or height_crop_num > 1:
                tokenized_image += ([self.image_token_id] * (num_queries * width_crop_num) + [self.image_token_id]) * (
                    num_queries * height_crop_num)

        else:  # crop_mode = False
            crop_ratio = (1, 1)
            images_spatial_crop.append([1, 1])

            # For smaller base sizes, resize; for larger, pad
            if self.base_size <= 640:
                resized_image = image.resize((self.base_size, self.base_size), Image.LANCZOS)
                images_list.append(self.image_transform(resized_image).to(self.dtype))
            else:
                global_view = ImageOps.pad(
                    image, (self.base_size, self.base_size),
                    color=tuple(int(x * 255) for x in self.image_transform.mean)
                )
                images_list.append(self.image_transform(global_view).to(self.dtype))

            num_queries = math.ceil((self.base_size // self.patch_size) / self.downsample_ratio)
            tokenized_image = ([self.image_token_id] * num_queries + [self.image_token_id]) * num_queries
            tokenized_image += [self.image_token_id]

        return images_list, images_crop_list, images_spatial_crop, tokenized_image, crop_ratio

    def process_single_sample(self, messages: List[Dict]) -> Dict[str, Any]:
            """
            Process a single conversation into model inputs.
            """

            # --- 1. Setup ---
            images = []
            for message in messages:
                if "images" in message and message["images"]:
                    for img_data in message["images"]:
                        if img_data is not None:
                            pil_image = self.deserialize_image(img_data)
                            images.append(pil_image)

            if not images:
                raise ValueError("No images found in sample. Please ensure all samples contain images.")

            tokenized_str = []
            images_seq_mask = []
            images_list, images_crop_list, images_spatial_crop = [], [], []

            prompt_token_count = -1 # Index to start training
            assistant_started = False
            image_idx = 0

            # Add BOS token at the very beginning
            tokenized_str.append(self.bos_id)
            images_seq_mask.append(False)

            for message in messages:
                role = message["role"]
                content = message["content"]

                # Check if this is the assistant's turn
                if role == "<|Assistant|>":
                    if not assistant_started:
                        # This is the split point. All tokens added *so far*
                        # are part of the prompt.
                        prompt_token_count = len(tokenized_str)
                        assistant_started = True

                    # Append the EOS token string to the *end* of assistant content
                    content = f"{content.strip()} {self.tokenizer.eos_token}"

                # Split this message's content by the image token
                text_splits = content.split('<image>')

                for i, text_sep in enumerate(text_splits):
                    # Tokenize the text part
                    tokenized_sep = text_encode(self.tokenizer, text_sep, bos=False, eos=False)
                    tokenized_str.extend(tokenized_sep)
                    images_seq_mask.extend([False] * len(tokenized_sep))

                    # If this text is followed by an <image> tag
                    if i < len(text_splits) - 1:
                        if image_idx >= len(images):
                            raise ValueError(
                                f"Data mismatch: Found '<image>' token but no corresponding image."
                            )

                        # Process the image
                        image = images[image_idx]
                        img_list, crop_list, spatial_crop, tok_img, _ = self.process_image(image)

                        images_list.extend(img_list)
                        images_crop_list.extend(crop_list)
                        images_spatial_crop.extend(spatial_crop)

                        # Add image placeholder tokens
                        tokenized_str.extend(tok_img)
                        images_seq_mask.extend([True] * len(tok_img))

                        image_idx += 1 # Move to the next image

            # --- 3. Validation and Final Prep ---
            if image_idx != len(images):
                raise ValueError(
                    f"Data mismatch: Found {len(images)} images but only {image_idx} '<image>' tokens were used."
                )

            # If we never found an assistant message, we're in a weird state
            # (e.g., user-only prompt). We mask everything.
            if not assistant_started:
                print("Warning: No assistant message found in sample. Masking all tokens.")
                prompt_token_count = len(tokenized_str)

            # Prepare image tensors
            images_ori = torch.stack(images_list, dim=0)
            images_spatial_crop_tensor = torch.tensor(images_spatial_crop, dtype=torch.long)

            if images_crop_list:
                images_crop = torch.stack(images_crop_list, dim=0)
            else:
                images_crop = torch.zeros((1, 3, self.base_size, self.base_size), dtype=self.dtype)

            return {
                "input_ids": torch.tensor(tokenized_str, dtype=torch.long),
                "images_seq_mask": torch.tensor(images_seq_mask, dtype=torch.bool),
                "images_ori": images_ori,
                "images_crop": images_crop,
                "images_spatial_crop": images_spatial_crop_tensor,
                "prompt_token_count": prompt_token_count, # This is now accurate
            }

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
        """Collate batch of samples"""
        batch_data = []

        # Process each sample
        for feature in features:
            try:
                processed = self.process_single_sample(feature['messages'])
                batch_data.append(processed)
            except Exception as e:
                print(f"Error processing sample: {e}")
                continue

        if not batch_data:
            raise ValueError("No valid samples in batch")

        # Extract lists
        input_ids_list = [item['input_ids'] for item in batch_data]
        images_seq_mask_list = [item['images_seq_mask'] for item in batch_data]
        prompt_token_counts = [item['prompt_token_count'] for item in batch_data]

        # Pad sequences
        input_ids = pad_sequence(input_ids_list, batch_first=True, padding_value=self.tokenizer.pad_token_id)
        images_seq_mask = pad_sequence(images_seq_mask_list, batch_first=True, padding_value=False)

        # Create labels
        labels = input_ids.clone()

        # Mask padding tokens
        labels[labels == self.tokenizer.pad_token_id] = -100

        # Mask image tokens (model shouldn't predict these)
        labels[images_seq_mask] = -100

        # Mask user prompt tokens when train_on_responses_only=True (only train on assistant responses)
        if self.train_on_responses_only:
            for idx, prompt_count in enumerate(prompt_token_counts):
                if prompt_count > 0:
                    labels[idx, :prompt_count] = -100

        # Create attention mask
        attention_mask = (input_ids != self.tokenizer.pad_token_id).long()

        # Prepare images batch (list of tuples)
        images_batch = []
        for item in batch_data:
            images_batch.append((item['images_crop'], item['images_ori']))

        # Stack spatial crop info
        images_spatial_crop = torch.cat([item['images_spatial_crop'] for item in batch_data], dim=0)

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
            "images": images_batch,
            "images_seq_mask": images_seq_mask,
            "images_spatial_crop": images_spatial_crop,
        }

### Preparing Trainer

In [ ]:
from transformers import Trainer, TrainingArguments
from unsloth import is_bf16_supported
FastVisionModel.for_training(model) # Enable for training!
data_collator = DeepSeekOCRDataCollator(
    tokenizer=tokenizer,
    model = model,
    image_size=640,
    base_size=1024,
    crop_mode=True,
    train_on_responses_only=True,
)

trainer = Trainer(
    model = model,
    tokenizer = tokenizer,
    data_collator = data_collator, # Must use!
    train_dataset = converted_dataset,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # max_steps = 60,
        num_train_epochs = 1, # Set this instead of max_steps for full training runs
        learning_rate = 1e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        fp16 = not is_bf16_supported(),  # Use fp16 if bf16 is not supported
        bf16 = is_bf16_supported(),  # Use bf16 if supported
        output_dir = "outputs",
        report_to = "none",     # For Weights and Biases
        dataloader_num_workers=2,
        # You MUST put the below items for vision finetuning:
        remove_unused_columns = False,
        save_steps = 200,
        save_total_limit = 2,
    ),
)

/tmp/ipykernel_20/326678009.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer._unsloth___init__`. Use `processing_class` instead.
  trainer = Trainer(


In [19]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
4.254 GB of memory reserved.


### Training

In [20]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 8,537 | Num Epochs = 1 | Total steps = 534
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 77,509,632 of 3,413,615,872 (2.27% trained)
Unsloth: Not an error, but DeepseekOCRForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([6, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
Unsloth: Will smartly offload gradients to save VRAM!
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([5, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([3, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([3, 100, 1280])


Step,Training Loss
1,1.798700
2,3.040400
3,2.352500
4,1.879700
5,2.229500
6,1.680900
7,1.539200
8,1.616900
9,1.311500
10,1.428500


BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([7, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([4, 100, 1280])
BASE: 

You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and can yield errors.


BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([5, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([3, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([5, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([3, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([3, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([3, 100, 1280])
BASE: 

You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and can yield errors.


BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([6, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([6, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([3, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([3, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([3, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([3, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([3, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([5, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([4, 100, 1280])
BASE: 

You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and can yield errors.


## 6. Evaluating Fine-tuned Model

In [21]:
FastVisionModel.for_inference(model) # Enable for inference!

finetuned_results = evaluate_model(
    model=model,
    tokenizer=tokenizer,
    dataset=eval_dataset,
    num_samples=200,
    base_size=1024,
    image_size=640,
    crop_mode=True,
    verbose=True
)

print_evaluation_summary(finetuned_results, "Fine-tuned Model Performance")
save_evaluation_results(finetuned_results, "finetuned_evaluation.json")


Evaluating:   0%|          | 0/200 [00:00<?, ?it/s]

BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])


Evaluating:   0%|          | 1/200 [00:06<20:15,  6.11s/it]

BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])


Evaluating:   1%|          | 2/200 [00:12<21:05,  6.39s/it]

...

BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([3, 100, 1280])


Evaluating:  56%|█████▋    | 113/200 [29:37<46:15, 31.90s/it]

BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([4, 100, 1280])


Evaluating: 100%|██████████| 200/200 [33:12<00:00,  9.96s/it]


Fine-tuned Model Performance
Number of samples: 200
Mean CER: 32.41%
Median CER: 12.62%
Std Dev: 87.92%
Min CER: 0.00%
Max CER: 1188.66%

 Best Predictions (Lowest CER):

Sample 9 (CER: 0.00%)
Reference:  gần và xa bờ, hợp thành phòng tuyến bảo vệ, kiểm soát và làm chủ...
Prediction: gần và xa bờ, hợp thành phòng tuyến bảo vệ, kiểm soát và làm chủ...

Sample 19 (CER: 0.00%)
Reference:  xã hội, có liên quan trực tiếp đến sự phồn vinh của đất nước, đến văn...
Prediction: xã hội, có liên quan trực tiếp đến sự phồn vinh của đất nước, đến văn...

Sample 205 (CER: 0.00%)
Reference:  Hà...
Prediction: Hà...

 Worst Predictions (Highest CER):

Sample 104 (CER: 1188.66%)
Reference:  Chúng ta " hoan nghênh và biểu dương những việc làm tốt của công nhân xây dựng trong việc thông tin,...
Prediction: Chúng tôi cơ hội nghiệp vụ du lịch chương trình văn phòng, chương trình văn phòng, chương trình văn ...

Sample 134 (CER: 150.00%)
Reference:  vụ...
Prediction: anh...

Sample 224 (CER: 100.00%)
Refer

## 7. Visualize Training Loss

In [ ]:
log_path = "/kaggle/working/outputs/checkpoint-534/trainer_state.json"

output_dir = "/kaggle/working/figures"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "training_loss_curve.png")

with open(log_path, "r") as f:
    data = json.load(f)

history = data.get("log_history", [])

steps = []
losses = []

for i, entry in enumerate(history):
    if "loss" in entry:
        losses.append(entry["loss"])
        steps.append(entry.get("step", i))

if len(losses) == 0:
    raise ValueError(" Không có loss trong trainer_state.json")

fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(steps, losses, linewidth=2)
ax.set_xlabel("Training Steps")
ax.set_ylabel("Loss")
ax.set_title("Training Loss Curve")
ax.grid(True)

# ===== SAVE FIG (FIG CHỨA AXES) =====
fig.savefig(output_path, dpi=300, bbox_inches="tight")
plt.close(fig)

print(f" Đã lưu ảnh training loss tại: {output_path}")
print(f" Số điểm loss: {len(losses)}")